# Experiment Runner: QRC-ESN vs. Classical ESN

### **Notebook Objective**

This notebook serves as the main script to run a series of experiments comparing two reservoir computing models:
1.  **Quantum Reservoir Computer (QRC-ESN)**
2.  **Classical Echo State Network (ESN)**

### **Process**
- Defines the experiment configuration (data profiles, hyperparameter grids, and constants).
- Iterates through each defined data profile.
- For each profile, it performs a grid search over the hyperparameter space for both models, using parallel processing (`joblib`) for efficiency.
- It saves all collated results into a single `.csv` file for later analysis in a separate notebook.

In [4]:
# === IMPORTS AND SETUP ===
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import our custom modules from the 'src' directory
from src.data_generation import mackey_glass, generate_arma_data, generate_narma_data
from src.experiment import run_qrc_experiment_with_subseeds, run_classical_experiment_with_subseeds

import itertools
import pandas as pd
from joblib import Parallel, delayed
from tqdm.notebook import tqdm


# Jupyter magic command for automatic reloading of external modules
%load_ext autoreload
%autoreload 2

print("Libraries and modules loaded successfully.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Libraries and modules loaded successfully.


In [5]:
# === EXPERIMENT CONFIGURATION ===

# General constants
SEED = 2025
TRAIN_FRACTION = 0.7
RESULTS_FILENAME = '../data/results_comparative.csv' # Results will be saved in the data folder

# Data profiles for analysis
data_profiles = [
    {'name': 'Mackey_Glass_(tau=17)', 'generator': mackey_glass, 'params': {'tau': 17}},
    {'name': 'Mackey_Glass_(tau=30)', 'generator': mackey_glass, 'params': {'tau': 30}},
    {'name': 'Mackey_Glass_(tau=100)', 'generator': mackey_glass, 'params': {'tau': 100}},
    {'name': 'ARMA_1_2_stochastic', 'generator': generate_arma_data, 'params': {}},
    {'name': 'NARMA10_Chaotic', 'generator': generate_narma_data, 'params': {'order': 10}},
    {'name': 'NARMA5_Chaotic', 'generator': generate_narma_data, 'params': {'order': 5}}
]

# Hyperparameter grid for the QRC-ESN model
param_grid_qrc = {
    'leakage_rate': [0.1, 0.3, 0.5, 0.7, 0.9],
    'lambda_reg': [1e-8],
    'window_size': [1, 2, 4, 6, 8, 10],
    'n_layers': [1, 2, 4],
    'lag': [0]
}

# Hyperparameter grid for the Classical ESN model
param_grid_classical = {
    'reservoir_size': [50, 100, 150],
    'spectral_radius': [0.9, 1.1, 1.25],
    'sparsity': [0.1, 0.2],
    'leakage_rate': [0.1, 0.3, 0.5, 0.7, 0.9],
    'lambda_reg': [1e-8]
}

print(f"Configuration ready. Results will be saved to: {RESULTS_FILENAME}")

Configuration ready. Results will be saved to: ../data/results_comparative.csv


In [6]:
# === MAIN EXECUTION LOOP ===

all_results = []

for profile in data_profiles:
    print(f"\n{'='*20}\nSTARTING PROFILE: {profile['name']}\n{'='*20}")
    time_series = profile['generator'](**profile['params'])
    
    # --- QRC Grid Search ---
    param_combinations_qrc = list(itertools.product(*param_grid_qrc.values()))
    qrc_results = Parallel(n_jobs=-1)(
        delayed(run_qrc_experiment_with_subseeds)(params, profile, time_series, TRAIN_FRACTION, SEED)
        for params in tqdm(param_combinations_qrc, desc=f"QRC Grid Search ({profile['name']})")
    )
    all_results.extend(filter(None, qrc_results))

    # --- Classical ESN Grid Search ---
    param_combinations_classical = list(itertools.product(*param_grid_classical.values()))
    classical_results = Parallel(n_jobs=-1)(
        delayed(run_classical_experiment_with_subseeds)(params, profile, time_series, TRAIN_FRACTION, SEED)
        for params in tqdm(param_combinations_classical, desc=f"Classical ESN Search ({profile['name']})")
    )
    all_results.extend(filter(None, classical_results))

print("\n--- ALL EXPERIMENTS COMPLETED ---")


STARTING PROFILE: Mackey_Glass_(tau=17)


QRC Grid Search (Mackey_Glass_(tau=17)):   0%|          | 0/90 [00:00<?, ?it/s]

Classical ESN Search (Mackey_Glass_(tau=17)):   0%|          | 0/90 [00:00<?, ?it/s]


STARTING PROFILE: Mackey_Glass_(tau=30)


QRC Grid Search (Mackey_Glass_(tau=30)):   0%|          | 0/90 [00:00<?, ?it/s]

...

Classical ESN Search (NARMA10_Chaotic):   0%|          | 0/90 [00:00<?, ?it/s]


STARTING PROFILE: NARMA5_Chaotic


QRC Grid Search (NARMA5_Chaotic):   0%|          | 0/90 [00:00<?, ?it/s]

Classical ESN Search (NARMA5_Chaotic):   0%|          | 0/90 [00:00<?, ?it/s]


--- ALL EXPERIMENTS COMPLETED ---


In [7]:
# === SAVE RESULTS ===

# Convert the list of dictionaries to a pandas DataFrame
results_df = pd.DataFrame(all_results)

# Save the DataFrame to a CSV file
results_df.to_csv(RESULTS_FILENAME, index=False)

print(f"Successfully saved {len(results_df)} results to {RESULTS_FILENAME}")
results_df.head()

Successfully saved 1080 results to ../data/results_comparative.csv


,model_type,data_profile,median_mse,std_dev_mse,cv_mse,leakage_rate,lambda_reg,window_size,n_layers,lag,base_seed,reservoir_size,spectral_radius,sparsity
0,QRC,Mackey_Glass_(tau=17),0.038000,9.587460e-18,2.523026e-16,0.1,1.000000e-08,1,1.0,0.0,2025,NaN,NaN,NaN
1,QRC,Mackey_Glass_(tau=17),0.038000,1.565626e-17,4.120084e-16,0.1,1.000000e-08,1,2.0,0.0,2025,NaN,NaN,NaN
2,QRC,Mackey_Glass_(tau=17),0.038000,1.434309e-17,3.774514e-16,0.1,1.000000e-08,1,4.0,0.0,2025,NaN,NaN,NaN
3,QRC,Mackey_Glass_(tau=17),0.001233,8.319067e-03,6.747859e+00,0.1,1.000000e-08,2,1.0,0.0,2025,NaN,NaN,NaN
4,QRC,Mackey_Glass_(tau=17),0.000696,2.761634e-04,3.965467e-01,0.1,1.000000e-08,2,2.0,0.0,2025,NaN,NaN,NaN
